In [ ]:
# Random forest classification algorithm from scratch using solar dataset

In [39]:
# Load csv file 
from csv import reader
def load_csv(filename):
    dataset=list()
    open_file=open(filename,'r')
    read_file=reader(open_file)
    for row in read_file:
        if not row:
            continue
        dataset.append(row)
    return dataset

In [40]:
# Convert string values to float values
def convert_str_to_float(dataset,column):
    for row in dataset:
        for i in range(len(row)):
            row[column]=float(row[column])

In [41]:
# Converting String value to integer value
def convert_str_to_int(dataset,column):
    class_values=[row[column] for row in dataset]
    unique=set(class_values)
    lookup=dict()
    for i,value in enumerate(unique):
        lookup[value]=i
    for row in dataset:
        row[column]=lookup[row[column]]
    return lookup

In [42]:
# Calculate minmax values for each column
def minmax(dataset):
    minmax=list()
    for i in range(len(dataset[0])):
        column_values=[row[i] for row in dataset]
        min_value=min(column_values)
        max_value=max(column_values)
        minmax.append([min_value,max_value])
    return minmax

In [43]:
# Apply normalize scale to dataset
def normalize_scale(dataset,minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i]=(row[i]-minmax[i[0]])/(minmax[i][1]-minmax[i][0])

In [44]:
# Accuracy of a metrics
def classification_accuracy(actual,predicted):
    correct=0
    for i in range(len(actual)):
        if actual[i]==predicted[i]:
            correct+=1
    return correct/float(len(actual))*100.0

In [45]:
# Split dataset by kfold crossvalidation technique
from random import randrange
def KFold(dataset,folds):
    fold_value=list()
    dataset_copy=list(dataset)
    fold_size=int(len(dataset)/folds)
    for _ in range(folds):
        fold=list()
        while len(fold)<fold_size:
            index=randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        fold_value.append(fold)
    return fold_value

In [46]:
# Evaluate Accuracy of model
def evaluate_model(dataset,algorithm,folds,*args):
    folds=KFold(dataset,folds)
    predictions=list()
    for fold in folds:
        train_set=list(folds)
        train_set.remove(fold)
        train_set=sum(train_set,[])
        test_set=list()
        for row in fold:
            row_copy=list(row)
            row_copy[-1]=None
            test_set.append(row_copy)
        predicted=algorithm(train_set,test_set,*args)
        actual=[row[-1] for row in fold]
        accuracy=classification_accuracy(actual,predicted)
        predictions.append(accuracy)
    return predictions

In [58]:
# Random Forest Algorithm 
def random_forest_algorithm(train,test,max_depth,min_size,sample_size,n_tree,n_features):
    trees=list()
    for _ in range(n_tree):
        # create a random subsample from the dataset with replacement
        sample=subsample(train,sample_size)
       # print("sample",sample)
        # Build a decision tree
        tree=build_tree(sample,max_depth,min_size,n_features)
        trees.append(tree)
    predictions=[bagging_predict(trees,row) for row in test]
    return predictions

In [59]:
# Create a random subsample from the dataset with replacement
def subsample(dataset,sample_size):
    sample=list()
    n_sample=round(len(dataset)*sample_size)
    #print("n_sample",n_sample)
    while len(sample)<n_sample:
        index=randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [60]:
# Build a decision tree
def build_tree(sample,max_depth,min_size,n_features):
    # Building Root of a decision tree
    root=root_split(sample,n_features)
   # print("root",root)
    # Create child splits for a node or make terminal
    split(root,max_depth,min_size,n_features,1)
    return root

In [61]:
# Building root of decision tree
def root_split(sample,n_features):
    # Getting class values 
    class_value=list(set(row[-1] for row in dataset))
   # print("class_value",class_value)
    # Default value setting
    b_index,b_value,b_score,b_groups=999,999,999,None
    features=list()
    while len(features)<n_features:
        index=randrange(len(dataset[0])-1)
     #   print("index",index)
        if index not in features:
            features.append(index)
       #     print("features.append(index)",features.append(index))
    for index in features:
        for row in dataset:
            # Split Dataset based on attribute value
            groups=test_split(index,row[index],dataset)
            # Calculate gini index for a split dataset.
            gini=gini_index(groups,class_value)
            if gini<b_score:
                b_index,b_value,b_score,b_groups=index,row[index],gini,groups
    return {'index':b_index,'value':b_value,'groups':b_groups}

In [62]:
# Split dataset based on attribute value
# row[index]=value
def test_split(index,value,dataset):
    left,right=list(),list()
    for row in dataset:
        if row[index]<value:
            left.append(row)
        else:
            right.append(row)
    return left,right

In [63]:
# Calculate gini index for a split dataset
def gini_index(groups,classes):
    n_instance=float(sum([len(group) for group in groups]))
   # print("n_instance",n_instance)
    gini=0.0
    for group in groups:
        size=float(len(group))
       # print("size",size)
        if size==0:
            continue
        score=0.0
        for class_val in classes:
            p=[row[-1] for row in group].count(class_val)/size
            score+=p*p
        gini+=(1.0-score)*(size/n_instance)
   # print("gini",gini)
    return gini

In [64]:
# Create child splits for a node or make terminal
def split(root,max_depth,min_size,n_features,depth):
    left,right=root['groups']
    del(root['groups'])
    # Check for a no split
    if not left or not right:
        root['left']=root['right']=to_terminal(left+right)
       # print("to_terminal(left+right)",to_terminal(left+right))
       # print("root['left']",root['left'])
       # print("root['right']",root['right'])
        return
    # check max depth
    if depth>=max_depth:
        root['left'],root['right']=to_terminal(left),to_terminal(right)
      #  print("check max depth root['left']",root['left'])
       # print("check max depth root['right']",root['right'])
        return
    # process left child
    if len(left)<=min_size:
        root['left']=to_terminal(left)
       # print("process left child root['left']",root['left'])
    else:
        root['left']=root_split(left,n_features)
        split(root['left'],max_depth,min_size,n_features,depth+1)
    # process right child
    if len(right)<=min_size:
        root['right']=to_terminal(right)
       # print("process left child root['right']",root['right'])
    else:
        root['right']=root_split(right,n_features)
        split(root['right'],max_depth,min_size,n_features,depth+1)

In [65]:
# Create a terminal node value
def to_terminal(group):
    outcomes=[row[-1] for row in group]
   # print("outcomes",outcomes)
   # print("max(set(outcomes),key=outcomes.count)",max(set(outcomes),key=outcomes.count))
    return max(set(outcomes),key=outcomes.count)

In [66]:
# Make a prediction with a list of bagged trees
def bagging_predict(trees,row):
    # Make predict a decision tress
    predictions=[predict(tree,row) for tree in trees]
   # print("predictions",predictions)
   # print("max(set(predictions),key=predictions.count)",max(set(predictions),key=predictions.count))
    return max(set(predictions),key=predictions.count)

In [67]:
# make predict a decision tree
def  predict(node,row):
    if row[node['index']]<node['value']:
       # print("row[node['index']]<node['value']",row[node['index']]<node['value'])
        if isinstance(node['left'],dict):
         #   print("predict(node['left'],row)",predict(node['left'],row))
            return predict(node['left'],row)
        else:
          #  print("node['left']",node['left'])
            return node['left']
    else:
        if isinstance(node['right'],dict):
           # print("predict(node['right'],row)",predict(node['right'],row))
            return predict(node['right'],row)
        else:
          #  print("node['right']",node['right'])
            return node['right']

In [69]:
# test the random forest algorithm on sonar dataset
from random import seed
from math import sqrt
seed(1)
# load and prepare file
filename='sonar.all-data.csv'
dataset=load_csv(filename)
for i in range(0,len(dataset[0])-1):
    convert_str_to_float(dataset,i)
convert_str_to_int(dataset,len(dataset[0])-1)
folds=5
max_depth=10
min_size=1
sample_size=1.0
n_features=int(sqrt(len(dataset[0])-1))
for n_trees in [1,5,10]:
    accuracy=evaluate_model(dataset,random_forest_algorithm,folds,max_depth,min_size,sample_size,n_trees,n_features)
    print("n_tree",n_trees)
    print("accuracy",accuracy)
    print("mean accuracy",sum(accuracy)/float(len(accuracy)))

('n_tree', 1)
('accuracy', [73.17073170731707, 75.60975609756098, 73.17073170731707, 85.36585365853658, 70.73170731707317])
('mean accuracy', 75.60975609756098)
('n_tree', 5)
('accuracy', [82.92682926829268, 70.73170731707317, 78.04878048780488, 80.48780487804879, 78.04878048780488])
('mean accuracy', 78.04878048780488)
('n_tree', 10)
('accuracy', [73.17073170731707, 82.92682926829268, 75.60975609756098, 92.6829268292683, 85.36585365853658])
('mean accuracy', 81.95121951219512)
